### 데이터 프레임 다른 포맷 형식으로 저장

<img src="https://pandas.pydata.org/docs/_images/02_io_readwrite.svg">

In [1]:
import pandas as pd

In [2]:
dict_data = {
    "c0":[1,2,3,],
    "c1":[4,5,6]
}
df = pd.DataFrame(dict_data)
df

,c0,c1
0,1,4
1,2,5
2,3,6


In [3]:
df.to_csv("./df1.csv",index=False)

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--start-maximized")
    # chrome_options.add_argument("--headless")   # 브라우저 띄우지 않고 결과값 받기
    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()), options=chrome_options
    )
    return driver

### Youtube 댓글 크롤링 후 데이터 프레임 생성

In [13]:
from bs4 import BeautifulSoup

In [18]:
import time

driver = set_chrome_driver()
driver.get("https://www.youtube.com/watch?v=HosW0gulISQ")

time.sleep(3)

# 현재 문서 높이
prev_height = driver.execute_script("return document.documentElement.scrollHeight")

# 스크롤 이동 시간
interval = 5

while True:

    # 스크롤 이동
    driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight)")

    # 페이지 로딩 대기
    time.sleep(interval)

    # 스크롤이 징행된 후 현재 문서 높이
    curr_height = driver.execute_script("return document.documentElement.scrollHeight")

    if curr_height == prev_height:
        break

    prev_height = curr_height

time.sleep(2)

soup = BeautifulSoup(driver.page_source, "lxml")

In [24]:
# 파싱 - id, 내용

author_list = soup.select("#author-text > span")
contents = soup.select("#content-text")

print(len(author_list), len(contents))


1266 1266


In [25]:
# 태그 제거

author_new = []
content_new = []

for i in range(len(author_list)):
    #작성자 가져오기
    author = author_list[i].text
    author = author.replace("\n","")
    author = author.replace("\t","")
    author = author.replace("      "," ")
    author_new.append(author)

    # 내용 가져오기
    content = contents[i].text
    content = content.replace("\n","")
    content = content.replace("\t","")
    content = content.replace("      "," ")
    content_new.append(content)

In [27]:
df = pd.DataFrame({"author":author_new, "comment":content_new})
df.head()

,author,comment
0,@user-vj4lt5vu2r,"""성공한 사람에게는 추억을""""성공할려는 자에게는 용기를""간략히자세히 보기"
1,@Jerjeon,와 진짜 영화 한장면 넣어놓은거 같음..전율이 미쳤음...와...소름돋는다간략히자...
2,@user-yz5ti1wp3u,데뷔할 때만 해도 20대였던 크리스와 멤버들..어느덧 벌써 40이 넘어가셨네..진...
3,@user-hd8vz7ww7w,"진짜 들을 때마다 매순간 빠짐없이 벅차오르는 개미친 노래 .., ㄹㅇ 죽기 전에는..."
4,@qjsdurwhk,10년 넘게 들어도 질리지 않는 명곡...간략히자세히 보기


In [ ]:
df.to_csv("./youtube.csv", index=False)

#### [참고] 전체 스크롤의 크기 확인하기

In [20]:
# body 태그
height = driver.execute_script("return document.body.scrollHeight")
print("body_scrollHeight",height)


# html 태그
doc_height = driver.execute_script("return document.documentElement.scrollHeight")
print(".documentElement_scrollHeight",doc_height)

body_scrollHeight 0
.documentElement_scrollHeight 143196
